
SVM tutorials
[youtube one](https://www.youtube.com/watch?v=0rjlviOQlbc)


# Distracted driver Detection

Implementation thorugh svm, Will be done in the following steps
1. Loading images and converting into arrays
2. Preprocessing such as rescaling,normalisation and train test split
3. Model selection,Training and hyper parameter tuning
4. Prediction, insights and accuracy calculation 



# Importing the libraries

In [ ]:
!pip install mahotas

     |████████████████████████████████| 5.7MB 5.0MB/s 


In [ ]:
# The import cell, Need to run this first for remaining ones to work
import shutil
import glob

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import cv2
import mahotas

from skimage import transform as tf
import progressbar
from time import sleep

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn import svm, metrics, datasets

from skimage import io 
from skimage.transform import resize
from PIL import Image
from pathlib import Path


from sklearn.utils import Bunch
from sklearn.model_selection import GridSearchCV, train_test_split
import skimage 

In [ ]:
import tensorflow as tf, tensorflow.keras.backend as K

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


### Mounting data into colab

Please follow the steps below to download and use kaggle data within Google Colab:

1. Go to your account, Scroll to API section and Click Expire API Token to remove previous tokens

2. Click on Create New API Token - It will download kaggle.json file on your machine.

3. Go to your Google Colab project file and run the following commands:


https://www.kaggle.com/tanyadayanand/end-to-end-pipeline-for-training-cnns-resnet

In [ ]:
! pip install -q kaggle

kaggle always mounts the older version run the below command to upgrade

or check [here](https://stackoverflow.com/questions/58643979/google-colaboratory-use-kaggle-server-version-1-5-6-client-version-1-5-4-fai)

In [ ]:
!pip install --upgrade --force-reinstall --no-deps kaggle

     |████████████████████████████████| 61kB 1.7MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.12-cp37-none-any.whl size=73053 sha256=73c0c7900ef1c294c5d847599e3669fb353df869a68330793e8ddfed0d6c2388
  Stored in directory: /root/.cache/pip/wheels/a1/6a/26/d30b7499ff85a4a4593377a87ecf55f7d08af42f0de9b60303
Successfully built kaggle
  Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12


Upload kaggle json file

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"midhunvarman","key":"ce3597e52e94bff107b80cc443b20bf8"}'}

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

To check if kaggle is mounted

In [ ]:
 ! kaggle datasets list

ref                                                         title                                              size  lastUpdated          downloadCount  voteCount  usabilityRating  
----------------------------------------------------------  ------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
gpreda/reddit-vaccine-myths                                 Reddit Vaccine Myths                              229KB  2021-05-21 06:23:33           5952        541  1.0              
crowww/a-large-scale-fish-dataset                           A Large Scale Fish Dataset                          3GB  2021-04-28 17:03:01           3327        287  0.9375           
imsparsh/musicnet-dataset                                   MusicNet Dataset                                   22GB  2021-02-18 14:12:19           1172        103  1.0              
dhruvildave/wikibooks-dataset                               Wikibooks Dataset             

# Importing distracted driver dataset



*   Shifting all the data to train and test folder




In [ ]:
! kaggle competitions download -c state-farm-distracted-driver-detection

100% 3.99G/4.00G [04:35<00:00, 48.3MB/s]
100% 4.00G/4.00G [04:35<00:00, 15.6MB/s]


Use this code to copy from colab to gdrive

In [ ]:
#to copy from colab to gdrive

import shutil
shutil.copy("/content/state-farm-distracted-driver-detection.zip", 
"/content/gdrive/MyDrive/sem 6/ml/project/state-farm-distracted-driver-detection.zip")

'/content/state-farm-distracted-driver-detection.zip'

## Loading Images

copy from gdrive to colab environment

In [ ]:
shutil.copy("/content/gdrive/MyDrive/sem 6/ml/project/state-farm-distracted-driver-detection.zip",
            "/content/state-farm-distracted-driver-detection.zip")

'/content/state-farm-distracted-driver-detection.zip'

unzip the following zip folder

In [ ]:
! unzip state-farm-distracted-driver-detection.zip -d state-farm-distracted-driver-detection

Streaming output truncated to the last 5000 lines.
  inflating: state-farm-distracted-driver-detection/imgs/train/c7/img_56636.jpg  
  inflating: state-farm-distracted-driver-detection/imgs/train/c7/img_56661.jpg  
  inflating: state-farm-distracted-driver-detection/imgs/train/c7/img_56699.jpg  
  inflating: state-farm-distracted-driver-detection/imgs/train/c7/img_56717.jpg  
  inflating: state-farm-distracted-driver-detection/imgs/train/c7/img_56724.jpg  
  inflating: state-farm-distracted-driver-detection/imgs/train/c7/img_56768.jpg  
  inflating: state-farm-distracted-driver-detection/imgs/train/c7/img_56779.jpg  
  inflating: state-farm-distracted-driver-detection/imgs/train/c7/img_56849.jpg  
  inflating: state-farm-distracted-driver-detection/imgs/train/c7/img_56899.jpg  
  inflating: state-farm-distracted-driver-detection/imgs/train/c7/img_56938.jpg  
  inflating: state-farm-distracted-driver-detection/imgs/train/c7/img_5706.jpg  
  inflating: state-farm-distracted-driver-detect

In [ ]:
import glob

#import directory to access files in a different path
datadir = "/content/state-farm-distracted-driver-detection/imgs/train/"
classes = ['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']
class_dict = {'c0' : 0, 'c1' : 0, 'c2' : 0, 'c3' : 0, 'c4' : 0, 'c5' : 0
              , 'c6' : 0, 'c7' : 0, 'c8' : 0, 'c9' : 0}

sum = 0
for x in classes:
  data = glob.glob(datadir + "/" + x + '/*.jpg')
  print(x,len(data))
  class_dict[x] = len(data)
  sum = sum + len(data)

print("total: ",sum)

c0 2489
c1 2267
c2 2317
c3 2346
c4 2326
c5 2312
c6 2325
c7 2002
c8 1911
c9 2129
total:  22424


In [ ]:
datadir = "/content/state-farm-distracted-driver-detection/imgs/train"

for c in classes:
  final_path = datadir +"/" +c
  bar = progressbar.ProgressBar(maxval=class_dict[c], \
    widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])
  with os.scandir(final_path) as entries:
    for count,entry in enumerate(entries):
      bar.update(count+1)
      shutil.copy(final_path + "/" + entry.name,
                  "/content/gdrive/MyDrive/sem 6/ml/project/state-farm-distracted-driver-detection/train/"+c +"/" + entry.name)

[========================================================================] 100%

## Preprocessing
We have 24000 images in training images each in either respective class folders from c0 to c9
And 79000 test images

### Feature extraction for svm

[global features](https://medium.com/@dataturks/understanding-svms-for-image-classification-cf4f01232700)

[Hu moments](https://learnopencv.com/shape-matching-using-hu-moments-c-python/)

[Historgram](https://docs.opencv.org/master/d1/db7/tutorial_py_histogram_begins.html)

[Haralick](http://murphylab.web.cmu.edu/publications/boland/boland_node26.html)

In [ ]:
# declaring a class for collecting the data information and data path is set 
class Configuration:
    def __init__(self):
        self.maxwidth =0
        self.maxheight=0
        self.minwidth = 35000
        self.minheight = 35000
        self.imgcount=0
        self.img_width_adjust = 480
        self.img_height_adjust= 360
        self.data_dir = "/content/state-farm-distracted-driver-detection/imgs/train"

#function for finding the max,min  heights and max,min widths. It traverses throught the all folders and count
#number of images in the entire data. Update the respective values in the config object accordingly. 
def calculateDimension(path):
    for subdir, dirs, files in os.walk(path):
        for file in files:
            if file.endswith(".jpg"):
                config.imgcount+=1
                filename = os.path.join(subdir, file)
                image = io.imread(filename)
                width = image.shape[0]
                height = image.shape[1]
                if width < config.minwidth:
                    config.minwidth = width
                if height < config.minheight:
                    config.minheight = height
                if width > config.maxwidth:
                    config.maxwidth = width
                if height > config.maxheight:
                    config.maxheight = height
    return

In [ ]:
#creating an config object 
config = Configuration()


calculateDimension(config.data_dir)
print("Minimum Width:\t",config.minwidth)
print("Minimum Height:\t",config.minheight)
print("Maximum Width:\t",config.maxwidth)
print("Maximum Height:\t",config.maxheight)
print("Image Count:\t",config.imgcount)

Minimum Width:	 480
Minimum Height:	 640
Maximum Width:	 480
Maximum Height:	 640
Image Count:	 22424


In [ ]:
fixed_size = tuple((64, 64))

#used global feature descriptors HuMoments functions for gathering the shape details
def fd_hu_moments(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # extract the shape feature vector
    feature = cv2.HuMoments(cv2.moments(image)).flatten()
    return feature

#used global feature descriptors haralick functions for gatahering the texture details
def fd_haralick(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # extract texture feature vector. This function expects the image to be in gray scale
    haralick = mahotas.features.haralick(gray).mean(axis=0)
    return haralick

#used global feature descriptors colorhistogram functions for gathering the color details
bins = 8
def fd_histogram(image, mask=None):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    #extract color feature vector
    hist  = cv2.calcHist([image], [0, 1, 2], None, [bins, bins, bins], [0, 256, 0, 256, 0, 256])
    cv2.normalize(hist, hist)
    return hist.flatten()

In [ ]:
driver_imgs_list = pd.read_csv('/content/state-farm-distracted-driver-detection/driver_imgs_list.csv')
# driver_imgs_list = pd.read_csv('/content/state-farm-distracted-driver-detection/driver_imgs_list.csv')

def find_person(image_path):
  global driver_imgs_list
  image_name = image_path.split("/")[-1]
  if(image_name.count('_') > 1):
    temp = image_name.split('_')
    image_name = temp[0] + '_' + temp[1] + '.jpg'
  person_details = driver_imgs_list.loc[driver_imgs_list["img"] == image_name]
  image_person = person_details["subject"].values[0]
  image_class = person_details["classname"].values[0][1:]
  return (image_person, image_class)

FileNotFoundError: ignored

In [ ]:

def load_image_files(container_path,list_path,dimension=(64, 64)):

    final_dataframe = pd.DataFrame(columns=['Image_name', 'Person', 'Vector', 'Class', "Global_features"])
    image_dir = Path(container_path)
    folders = [directory for directory in image_dir.iterdir() if directory.is_dir()]
    categories = [fo.name for fo in folders]
    
    print("cat",categories)
    print("fol",folders)
    print("dir",image_dir)
    
    images = []
    flat_data = []
    target = []
    global_features = []
    person = []
    for i, direc in enumerate(folders):
        # get the current training label
        current_label = direc.name
        i = 0
        for file in direc.iterdir():
            if(i < 1):
              print("file1",file)
            i = i+ 1
            image_person, image_class = find_person(str(file))
            file = str(file)
            image = cv2.imread(file)
            #resizing the image to 64*64
            image = cv2.resize(image, dimension)
            target.append(current_label)
            # Global feature extraction is done here 
            fv_hu_moments = fd_hu_moments(image)
            fv_haralick   = fd_haralick(image)
            fv_histogram  = fd_histogram(image)
            #combing the global feautures  
            global_feature = np.hstack([fv_histogram, fv_haralick, fv_hu_moments])
            global_features.append(global_feature)

            person.append(image_person)

            final_dataframe = final_dataframe.append({'Image_name': file, 
                                            "Person": image_person, 
                                            "Vector": image.flatten(), 
                                            "Class":current_label,
                                            "Global_features": global_feature}
                                            ,ignore_index=True)
        print(current_label)
        print("feature extraction done with folder: {}".format(current_label))

    print("Entire Feature Extraction done")
    return global_features,target,person,final_dataframe

In [ ]:
global_features,labels,peoples,final_dataframe = load_image_files("/content/state-farm-distracted-driver-detection/imgs/train",
                                          '/content/state-farm-distracted-driver-detection/driver_imgs_list.csv')

cat ['c7', 'c6', 'c0', 'c2', 'c1', 'c8', 'c5', 'c4', 'c9', 'c3']
fol [PosixPath('/content/state-farm-distracted-driver-detection/imgs/train/c7'), PosixPath('/content/state-farm-distracted-driver-detection/imgs/train/c6'), PosixPath('/content/state-farm-distracted-driver-detection/imgs/train/c0'), PosixPath('/content/state-farm-distracted-driver-detection/imgs/train/c2'), PosixPath('/content/state-farm-distracted-driver-detection/imgs/train/c1'), PosixPath('/content/state-farm-distracted-driver-detection/imgs/train/c8'), PosixPath('/content/state-farm-distracted-driver-detection/imgs/train/c5'), PosixPath('/content/state-farm-distracted-driver-detection/imgs/train/c4'), PosixPath('/content/state-farm-distracted-driver-detection/imgs/train/c9'), PosixPath('/content/state-farm-distracted-driver-detection/imgs/train/c3')]
dir /content/state-farm-distracted-driver-detection/imgs/train
file1 /content/state-farm-distracted-driver-detection/imgs/train/c7/img_2426.jpg
c7
feature extraction done

In [ ]:
#encoding the target labels 
targetNames = np.unique(labels)
le = LabelEncoder()
target = le.fit_transform(labels)
print("training labels encoded")
print(target)

training labels encoded
[7 7 7 ... 3 3 3]


In [ ]:
print(np.unique(target))

[0 1 2 3 4 5 6 7 8 9]


Reading the pickle file


In [ ]:
# add pickle file reading
print("feature vector size {}".format(np.array(global_features).shape))

feature vector size (22424, 532)


In [ ]:
svm_df = pd.DataFrame(global_features,columns=[x for x in range(0,532)])
print(svm_df)

            0         1         2    ...       529       530       531
0      0.074258  0.026706  0.000000  ...  0.084399  0.913401  0.996934
1      0.109427  0.149215  0.013228  ...  0.083960  0.906715  0.997047
2      0.167579  0.087052  0.027010  ...  0.083723  0.908656  0.997043
3      0.118264  0.079698  0.037093  ...  0.083993  0.913185  0.994169
4      0.108358  0.009128  0.008497  ...  0.085120  0.918172  0.995408
...         ...       ...       ...  ...       ...       ...       ...
22419  0.382485  0.153823  0.026847  ...  0.082362  0.907601  0.990550
22420  0.109975  0.140623  0.008181  ...  0.083961  0.906960  0.997048
22421  0.245829  0.054176  0.007204  ...  0.085176  0.904028  0.996566
22422  0.182880  0.057000  0.017686  ...  0.083700  0.911891  0.996583
22423  0.114247  0.052110  0.040422  ...  0.083519  0.910325  0.995828

[22424 rows x 532 columns]


In [ ]:
# Using 'Address' as the column name
# and equating it to the list
svm_df['labels'] = target
svm_df['person'] = peoples
# svm_df['target'] = target
print(svm_df)
svm_df.to_pickle('./svm_dataframe_new.pkl')

              0         1         2  ...       531  labels  person
0      0.074258  0.026706  0.000000  ...  0.996934       7    p045
1      0.109427  0.149215  0.013228  ...  0.997047       7    p047
2      0.167579  0.087052  0.027010  ...  0.997043       7    p014
3      0.118264  0.079698  0.037093  ...  0.994169       7    p026
4      0.108358  0.009128  0.008497  ...  0.995408       7    p049
...         ...       ...       ...  ...       ...     ...     ...
22419  0.382485  0.153823  0.026847  ...  0.990550       3    p026
22420  0.109975  0.140623  0.008181  ...  0.997048       3    p047
22421  0.245829  0.054176  0.007204  ...  0.996566       3    p066
22422  0.182880  0.057000  0.017686  ...  0.996583       3    p016
22423  0.114247  0.052110  0.040422  ...  0.995828       3    p026

[22424 rows x 534 columns]


In [ ]:
import shutil
shutil.copy("/content/svm_dataframe_new.pkl",
            "/content/gdrive/MyDrive/sem 6/ml/project/state-farm-distracted-driver-detection/svm_dataframe_new.pkl")

'/content/gdrive/MyDrive/sem 6/ml/project/state-farm-distracted-driver-detection/svm_dataframe_new.pkl'

In [ ]:
import shutil
shutil.copy("/content/gdrive/MyDrive/sem 6/ml/project/state-farm-distracted-driver-detection/svm_dataframe_new.pkl",
            "/content/svm_dataframe_new.pkl",)
svm_df = pd.read_pickle("./svm_dataframe_new.pkl")
print(svm_df)

              0         1         2  ...           531  labels  person
0      0.040589  0.004683  0.000000  ... -5.120664e-21       7    p045
1      0.059812  0.026168  0.002492  ...  2.702187e-22       7    p047
2      0.091597  0.015266  0.005089  ...  6.774413e-23       7    p014
3      0.064642  0.013977  0.006988  ... -1.362877e-19       7    p026
4      0.059228  0.001601  0.001601  ... -7.750927e-20       7    p049
...         ...       ...       ...  ...           ...     ...     ...
22419  0.209063  0.026976  0.005058  ... -3.080301e-19       3    p026
22420  0.060111  0.024661  0.001541  ...  2.959350e-22       3    p047
22421  0.134368  0.009501  0.001357  ... -2.256635e-20       3    p066
22422  0.099961  0.009996  0.003332  ... -2.175761e-20       3    p016
22423  0.062446  0.009138  0.007615  ... -5.758765e-20       3    p026

[22424 rows x 534 columns]


### Train validation split

without data leakage

In [ ]:
X_train,X_val = train_test_split(svm_df, train_size=0.9)

print(X_train)
print(X_val)

              0         1         2  ...           531  labels  person
11688  0.068629  0.004902  0.001634  ... -4.761649e-20       8    p021
6158   0.021751  0.001554  0.001554  ...  1.769840e-21       0    p012
15344  0.171410  0.017141  0.011999  ... -2.143379e-20       5    p024
8073   0.035644  0.009241  0.001320  ... -9.196018e-21       2    p022
3779   0.044097  0.004562  0.000000  ... -7.604158e-20       6    p026
...         ...       ...       ...  ...           ...     ...     ...
11736  0.073122  0.008705  0.001741  ... -2.770352e-20       8    p035
1244   0.043856  0.021116  0.003249  ... -7.415349e-20       7    p026
2533   0.074720  0.011623  0.000000  ... -1.670827e-20       6    p081
21744  0.138459  0.020307  0.018461  ... -3.448539e-20       3    p021
5900   0.098153  0.000000  0.000000  ... -1.199888e-20       0    p051

[20181 rows x 534 columns]
              0         1         2  ...           531  labels  person
14381  0.041075  0.004739  0.000000  ... -9.16322

In [ ]:
X_train_data = X_train.drop(columns=["labels","person"])
Y_train_data = X_train["labels"]
X_test_data = X_val.drop(columns=["labels","person"])
Y_test_data = X_val["labels"]


to avoid data leakage normalising after train_test split

In [ ]:
# normalize the feature vector in the range (0-1)
scaler = MinMaxScaler(feature_range=(0, 1))
X_train_data = scaler.fit_transform(X_train_data)
print("feature vector normalized")

print("target labels: {}".format(Y_train_data))
print("target labels shape: {}".format(np.array(Y_train_data).shape))

# get the overall feature vector size
print("feature vector size {}".format(np.array(X_train_data).shape))

feature vector normalized
target labels: 11688    8
6158     0
15344    5
8073     2
3779     6
        ..
11736    8
1244     7
2533     6
21744    3
5900     0
Name: labels, Length: 20181, dtype: int64
target labels shape: (20181,)
feature vector size (20181, 532)


In [ ]:
# normalize the feature vector in the range (0-1)
scaler = MinMaxScaler(feature_range=(0, 1))
X_test_data = scaler.fit_transform(X_test_data)
print("feature vector normalized")

print("target labels: {}".format(Y_test_data))
print("target labels shape: {}".format(np.array(Y_test_data).shape))

# get the overall feature vector size
print("feature vector size {}".format(np.array(X_test_data).shape))

feature vector normalized
target labels: 14381    5
3991     6
20106    3
4393     0
21196    3
        ..
16040    4
2127     6
13279    8
3919     6
15175    5
Name: labels, Length: 2243, dtype: int64
target labels shape: (2243,)
feature vector size (2243, 532)


### Training the model 

expected time to run using raw vectors were 4hrs mins 

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV


from sklearn.svm import SVC


# defining parameter range
param_grid = {"C": [0.001, 0.01, 0.1, 1, 10]
              ,"gamma": [0.001, 0.01, 0.1, 1, 10]
              ,"kernel": ['rbf']} 
  
# grid = GridSearchCV(SVC(), param_grid,n_jobs=-1, refit = True, verbose = 3)
grid_random = RandomizedSearchCV(SVC(), param_grid,n_jobs=-1, refit = True, verbose = 3)
  
# fitting the model for grid search
grid_random.fit(X_train_data, Y_train_data)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed: 120.2min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed: 222.8min finished


RandomizedSearchCV(cv=None, error_score=nan,
                   estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                                 class_weight=None, coef0=0.0,
                                 decision_function_shape='ovr', degree=3,
                                 gamma='scale', kernel='rbf', max_iter=-1,
                                 probability=False, random_state=None,
                                 shrinking=True, tol=0.001, verbose=False),
                   iid='deprecated', n_iter=10, n_jobs=-1,
                   param_distributions={'C': [0.001, 0.01, 0.1, 1, 10],
                                        'gamma': [0.001, 0.01, 0.1, 1, 10],
                                        'kernel': ['rbf']},
                   pre_dispatch='2*n_jobs', random_state=None, refit=True,
                   return_train_score=False, scoring=None, verbose=3)

In [ ]:
print("Best parameters", end=" : ")
print(grid_random.best_params_)
print("Best accuracy", end=" : ")
print(round(grid_random.best_score_ * 100, 2))

Best parameters : {'kernel': 'rbf', 'gamma': 1, 'C': 1}
Best accuracy : 97.54


In [ ]:
grid_cv = GridSearchCV(SVC(), param_grid,n_jobs=-1, refit = True, verbose = 3)
# grid = RandomizedSearchCV(SVC(), param_grid,n_jobs=-1, refit = True, verbose = 3)
  
# fitting the model for grid search
grid_cv.fit(X_train_data, Y_train_data)

In [ ]:
print("Best parameters", end=" : ")
print(grid_cv.best_params_)
print("Best accuracy", end=" : ")
print(round(grid_cv.best_score_ * 100, 2))

25 mins

In [ ]:
clf = svm.SVC(C = 1, gamma=1,kernel='rbf')
clf.fit(X_train_data,Y_train_data)

In [ ]:
Y_pred = clf.predict(X_test_data)

In [ ]:
from sklearn import metrics

print(metrics.classification_report(Y_test_data, Y_pred))
print("Accuracy Score  : {}".format(accuracy_score(Y_test_data, Y_pred)))

              precision    recall  f1-score   support

           0       0.99      0.68      0.80       254
           1       0.97      0.68      0.80       234
           2       1.00      0.63      0.77       244
           3       0.20      0.99      0.33       212
           4       1.00      0.65      0.79       228
           5       0.99      0.63      0.77       237
           6       0.98      0.71      0.82       255
           7       1.00      0.34      0.50       200
           8       1.00      0.33      0.49       175
           9       1.00      0.51      0.68       204

    accuracy                           0.62      2243
   macro avg       0.91      0.61      0.68      2243
weighted avg       0.92      0.62      0.69      2243

Accuracy Score  : 0.623272403031654
